In [1]:
#optimization: label_num = 2
#model: microsoft/CodeBERT
#learning rate: 1e-5
#train_test_split=0.1, seed = 100
#batch_size = 

In [2]:
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 7.8 MB/s 
     |████████████████████████████████| 86 kB 5.6 MB/s 
     |████████████████████████████████| 6.6 MB 50.1 MB/s 
     |████████████████████████████████| 596 kB 62.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 9.5 MB/s 
     |████████████████████████████████| 1.1 MB 58.5 MB/s 
     |████████████████████████████████| 212 kB 65.7 MB/s 
     |████████████████████████████████| 140 kB 72.4 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 127 kB 73.7 MB/s 
     |████████████████████████████████| 271 kB 56.5 MB/s 
     |████████████████████████████████| 94 kB 1.1 MB/s 
     |████████████████████████████████| 144 kB 3.8 MB/s 
     |████████████████████████████████| 112 kB 57.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.5.1
    Uninstalling dill-0.3.5.1:
      Successfully uninstalled dill-0.3.5.1
  Attempting unins

In [8]:
import pandas as pd
import re
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, RobertaTokenizer, RobertaConfig, RobertaModel
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss

In [6]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
#Load Train dataset
train1 = pd.read_csv('sample_train.csv', encoding = 'utf-8')
df1 = pd.DataFrame(train1)

In [10]:
#Define Model
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base")
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
#loss_fn = 

batch_size = 16
epoch_num = 10
MAX_LEN = 512
learning_rate = 1e-5

Downloading:   0%|          | 0.00/498 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be 

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [11]:
def preprocess(df):
  #preprocess_df = df.replace(re.compile('(^import.*|^from.*)',re.MULTILINE),"",regex=True) #import,from 없애기
  preprocess_df = df.replace(re.compile('(#.*)', re.MULTILINE),"",regex=True) #주석 한 줄
  preprocess_df = preprocess_df.replace(re.compile('[\'\"]{3}.*?[\'\"]{3}', re.DOTALL),"",regex=True) #주석 여러줄
  preprocess_df = preprocess_df.replace(re.compile('[\n]{2,}', re.MULTILINE),"\n",regex=True) #다중개행 한번으로
  preprocess_df = preprocess_df.replace(re.compile('[ ]{4}', re.MULTILINE),"\t",regex=True) #tab 변환
  preprocess_df = preprocess_df.replace(re.compile('[ ]{1,3}', re.MULTILINE)," ",regex=True) #공백 여러개 변환
  preprocess_df.to_csv('preprocess.csv')
  
def tokenized(examples):
  return tokenizer(examples['code1'],examples['code2'], padding=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [12]:
preprocess(df1)
dataset = load_dataset("csv",data_files="preprocess.csv")['train']
#truncation을 하는 대신에 왼쪽부터 자르도록 해야하나?
encoded_dataset = dataset.map(tokenized,remove_columns=['Unnamed: 0','code1','code2'],batched=True)
encoded_dataset=encoded_dataset.rename_column(original_column_name='similar',new_column_name='labels')
encoded_dataset
print(encoded_dataset[1])

Using custom data configuration default-4ce515a0bd9644d3


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-4ce515a0bd9644d3/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?ba/s]

{'labels': 0, 'input_ids': [0, 428, 6, 740, 5457, 5456, 1640, 2544, 6, 8135, 49123, 44154, 49338, 50118, 17265, 1640, 428, 1009, 740, 43, 2, 2, 41975, 295, 35187, 25, 46446, 50118, 282, 5457, 6979, 1640, 46797, 49338, 50118, 102, 5457, 46446, 4, 30766, 1640, 8458, 1640, 32557, 1640, 2544, 6, 46797, 49123, 44154, 43048, 47619, 50118, 1253, 5457, 46446, 4, 48204, 2723, 1640, 102, 43, 50118, 17265, 49570, 1253, 2055, 112, 43, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [13]:
print(tokenizer.tokenize(dataset['code2'][1]))

['import', 'Ġn', 'umpy', 'Ġas', 'Ġnp', 'Ċ', 'n', 'Ġ=', 'Ġint', '(', 'input', '())', 'Ċ', 'a', 'Ġ=', 'Ġnp', '.', 'array', '(', 'list', '(', 'map', '(', 'int', ',', 'input', '().', 'split', '()', ')))', 'Ċ', 'ans', 'Ġ=', 'Ġnp', '.', 'args', 'ort', '(', 'a', ')', 'Ċ', 'print', '(*', 'ans', 'Ġ+', 'Ġ1', ')']


In [14]:
encoded_dataset = encoded_dataset.train_test_split(0.1,seed=100)

In [13]:
#loss_fn = CrossEntropyLoss()#weight?

In [67]:
'''
class CodeBERTModified():
  def __init__(self, my_pretrained_model):
    super(CodeBERTModified,self).__init__()
    self.pretrained = my_pretrained_model
    self.parameters = self.pretrained.parameters
    #self.grads = self.pretrained.grads
    self.cache = None
  
  def forward(self,input_ids=None,token_type_ids=None,labels=None):
    outputs = self.pretrained(input_ids, token_type_ids=token_type_ids, labels=labels)
    print(outputs)
    out = outputs[1]#의미하는게 뭐지?
    loss = loss_fn(outputs[1].view(-1,2),labels.view(-1))#여기도 어떻게 돌아가는건지
    return out,loss

  def backward(self):
    return 

  def predict(self,input_ids=None,token_type_ids=None,labels=None):
    return self.forward(input_ids, token_type_ids=token_type_ids, labels=labels)
  '''

In [68]:
'''
model = CodeBERTModified(my_pretrained_model=pretrained)
model.to(device)
'''

In [32]:
from transformers import AdamW
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.9), eps=1e-8, 
            weight_decay=0.0)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [ ]:
'''
def flat_accuracy(preds,labels): #이거는 metric accuracy로 대체 가능할듯
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)
'''

In [25]:
'''
for epoch in range(epoch_num):

  #train
  print("======Epoch%d======",epoch)
  print("Traing...")

  #cuda memory error
  gc.collect()
  torch.cuda.empty_cache()

  total_train_loss = 0
  #model.train()

  for step, batch in enumerate(encoded_dataset['train']):
    label = batch['labels']
    input_ids = batch['input_ids']
    token_ids = batch['token_type_ids']
    
    #model.zero_grads()

    logits, loss = model.forward(input_ids = input_ids, token_type_ids = token_ids, labels=label)
    total_train_loss += loss.item()
    loss.backward()
    optimizer.step()

    if step % 50 == 0 and not step == 0:
      print('step : {:>5,} of {:>5,} loss: {:.5f}'.format(step, len(encoded_dataset['train']), loss.item()))

    avg_train_loss = total_train_loss / len(encoded_dataset['train'])
    print("")
    print("  Average training loss: {0:.5f}".format(avg_train_loss))


    #validation
    print("Validation...")

    model.eval()
    total_eval_accuracy = 0
    total_eval_loss=0

    for step,batch in enumerate(encoded_dataset['test']):
      label = batch[0]
      input_ids = batch[1]
      token_ids = batch[2]

      with torch.no_grads():
        logits, loss = model(input_ids = input_ids, token_type_ids = token_ids, labels=label)
        total_eval_loss += loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = label.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)


      avg_val_accuracy = total_eval_accuracy / len(encoded_dataset['test'])
      print("Accuracy: {0:.2f}".format(avg_val_accuracy))
  '''

'\nfor epoch in range(epoch_num):\n\n  #train\n  print("======Epoch%d======",epoch)\n  print("Traing...")\n\n  #cuda memory error\n  gc.collect()\n  torch.cuda.empty_cache()\n\n  total_train_loss = 0\n  #model.train()\n\n  for step, batch in enumerate(encoded_dataset[\'train\']):\n    label = batch[\'labels\']\n    input_ids = batch[\'input_ids\']\n    token_ids = batch[\'token_type_ids\']\n    \n    #model.zero_grads()\n\n    logits, loss = model.forward(input_ids = input_ids, token_type_ids = token_ids, labels=label)\n    total_train_loss += loss.item()\n    loss.backward()\n    optimizer.step()\n\n    if step % 50 == 0 and not step == 0:\n      print(\'step : {:>5,} of {:>5,} loss: {:.5f}\'.format(step, len(encoded_dataset[\'train\']), loss.item()))\n\n    avg_train_loss = total_train_loss / len(encoded_dataset[\'train\'])\n    print("")\n    print("  Average training loss: {0:.5f}".format(avg_train_loss))\n\n\n    #validation\n    print("Validation...")\n\n    model.eval()\n    tot

In [26]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [27]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred): #이게 필요한가?
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return metric.compute(predictions=predictions, references=labels)

metric_name = "accuracy"


In [ ]:
class TrainerModified(Trainer):
  def compute_loss(self,model,inputs,return_outputs=False):
    labels = inputs.get("labels")
    outputs = model()

In [35]:
args = TrainingArguments("test", save_strategy="epoch",evaluation_strategy="epoch",logging_strategy="epoch", 
                         learning_rate=learning_rate,per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,num_train_epochs=epoch_num,weight_decay=0.01,
                         do_train=True,do_eval=True,metric_for_best_model=metric_name,load_best_model_at_end=True)

trainer = Trainer(model,args,train_dataset=encoded_dataset['train'],eval_dataset=encoded_dataset['test'],
                  tokenizer=tokenizer, compute_metrics=compute_metrics, data_collator=data_collator,
                  optimizers=optimizers)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [36]:
gc.collect()
torch.cuda.empty_cache()

In [37]:
trainer.train()

***** Running training *****
  Num examples = 16173
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 10110


Epoch,Training Loss,Validation Loss,Accuracy
1,0.493900,0.268434,0.885921
2,0.224600,0.314565,0.904285
3,0.157800,0.137720,0.961603
4,0.126100,0.136520,0.969393
5,0.086100,0.231458,0.958264
6,0.081400,0.098523,0.979967
7,0.054400,0.173784,0.973289
8,0.048500,0.195078,0.971619
9,0.037500,0.169191,0.977741
10,0.027300,0.218782,0.970506


***** Running Evaluation *****
  Num examples = 1797
  Batch size = 16
Saving model checkpoint to test/checkpoint-1011
Configuration saved in test/checkpoint-1011/config.json
Model weights saved in test/checkpoint-1011/pytorch_model.bin
tokenizer config file saved in test/checkpoint-1011/tokenizer_config.json
Special tokens file saved in test/checkpoint-1011/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1797
  Batch size = 16
Saving model checkpoint to test/checkpoint-2022
Configuration saved in test/checkpoint-2022/config.json
Model weights saved in test/checkpoint-2022/pytorch_model.bin
tokenizer config file saved in test/checkpoint-2022/tokenizer_config.json
Special tokens file saved in test/checkpoint-2022/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1797
  Batch size = 16
Saving model checkpoint to test/checkpoint-3033
Configuration saved in test/checkpoint-3033/config.json
Model weights saved in test/checkpoint-3033/pytorch_mod

TrainOutput(global_step=10110, training_loss=0.13374870292746585, metrics={'train_runtime': 8840.6175, 'train_samples_per_second': 18.294, 'train_steps_per_second': 1.144, 'total_flos': 4.25529509833728e+16, 'train_loss': 0.13374870292746585, 'epoch': 10.0})

In [38]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1797
  Batch size = 16


{'epoch': 10.0,
 'eval_accuracy': 0.9799666110183639,
 'eval_loss': 0.09852267801761627,
 'eval_runtime': 32.5823,
 'eval_samples_per_second': 55.153,
 'eval_steps_per_second': 3.468}

In [39]:
TEST = "test.csv"
SAMPLE = "sample_submission.csv"

test_dataset = load_dataset("csv",data_files = TEST)['train']
test_dataset = test_dataset.map(tokenized,remove_columns=['code1','code2'])

predictions = trainer.predict(test_dataset)

df = pd.read_csv(SAMPLE)
df['similar'] = np.argmax(predictions.predictions,axis=-1)
df.to_csv('submission.csv',index=False)

Using custom data configuration default-d9bcc739a55afe8d


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-d9bcc739a55afe8d/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/179700 [00:00<?, ?ex/s]

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: pair_id. If pair_id are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 179700
  Batch size = 16
